# Домашнее задание 3
### Линейная и логистическая регрессия

Оформление дз:
Присылайте выполненное на почту `mlphystech@gmail.com` файлы .pdf и .ipynb или ссылку на свой гитхаб.

**Дедлайн: 11 апреля 23:59**

*Зуева Надежда*

ФИВТ МФТИ

Это задание будет посвящено линейной и логистической регрессии.
Мы надеемся, что вы уже знаете, что такое `kaggle`, так как сегодня нам предстоит поработать с самым *классическим* датасетом машинного обучения, а именно $-$ с Титаником!


Цель соревнования $-$ предсказать, кто из пассажиров выживет, а кто, увы, нет.

https://www.kaggle.com/c/titanic

## Теоретические вопросы
### 2 балла (по 0.5 за каждый верный ответ)
1. Что такое линейные модели классификации и регрессии? В чем их основные свойства? Где хорошо работают?
2. Что такое регуляризация и как она используется? 
3. Что такое MSE? Приведите пример.
4. Что такое градиентный спуск? Опишите базовый алгоритм работы.

P.S. просьба не копипастить из Википедии. Это все легко вычисляется и будет караться :)

## Задание 1 (4 балла)
Скачайте файлы `train.csv` и `test.csv`.

Выведите первые 10 строк из `train.csv`

In [65]:
from copy import deepcopy as dc
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error


data = pd.read_csv("./train.csv") #считайте данные из файла train.csv


In [66]:
#выведите первые 10 строк из data
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


Как мы видим, не все данные *

полны*. Например, это видно в столбце `Age`. 
В качестве простейшего решения мы предлагаем заменить пропущенные значения на медианные (функция `numpy.median`)
Вы можете воспользоваться нашим методом или произвести собственный анализ и заменить значения более умным способом (profit: бонусные баллы).

In [67]:
# Replace all the missing ages in the data with the median age
df = dc(data)
df["Age"] = df["Age"].fillna(df["Age"].median())

Теперь займемся **оцифровкой признаков**

Кстати, напомните, а зачем нам это нужно? :)

$*$ ваш ответ $*$

Какой параметр здесь бинарный? Оцифруйте его при помощи нулей и единиц.

# Ответ

** Потому что с числами удобнее работать, чем со строками **

In [68]:
df.loc[df["Sex"] == "male", "Sex"] = 0 #тут что-то должно быть
df.loc[df["Sex"] == "female", "Sex"] = 1 #тут что-то должно быть

Выведите описание нашего датасета после изменений с помощью функции `describe`.
Сколько процентов пассажиров выжило? Каков был средний возраст всех пассажиров?

In [69]:
print(df.describe())
# df.head(10)

       PassengerId    Survived      Pclass         Sex         Age  \
count   891.000000  891.000000  891.000000  891.000000  891.000000   
mean    446.000000    0.383838    2.308642    0.352413   29.361582   
std     257.353842    0.486592    0.836071    0.477990   13.019697   
min       1.000000    0.000000    1.000000    0.000000    0.420000   
25%     223.500000    0.000000    2.000000    0.000000   22.000000   
50%     446.000000    0.000000    3.000000    0.000000   28.000000   
75%     668.500000    1.000000    3.000000    1.000000   35.000000   
max     891.000000    1.000000    3.000000    1.000000   80.000000   

            SibSp       Parch        Fare  
count  891.000000  891.000000  891.000000  
mean     0.523008    0.381594   32.204208  
std      1.102743    0.806057   49.693429  
min      0.000000    0.000000    0.000000  
25%      0.000000    0.000000    7.910400  
50%      0.000000    0.000000   14.454200  
75%      1.000000    0.000000   31.000000  
max      8.000000

Продолжим заполнять пропуски. Рассмотрим графу пунктов отправления `Embarked` 

Там есть типы данных:

C = Cherbourg, Q = Queenstown, S = Southampton, а также `nan`

In [70]:
print(df["Embarked"].unique())
print(df['Survived'].mean())
print(df['Age'].mean())

['S' 'C' 'Q' nan]
0.383838383838
29.3615824916


In [71]:
import collections
print(collections.Counter(df['Embarked']))

Counter({'S': 644, 'C': 168, 'Q': 77, nan: 2})


С учетом последней ячейки заполните пропущенные значения по своему усмотрению. Объясните свой выбор, затем оцифруйте признак `Embarked` 

Например, заменив все на {0,1,2} или {12,42,10}, как хотите :)

In [72]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",0,28.0,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",0,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",0,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",1,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",1,14.0,1,0,237736,30.0708,NaN,C


In [73]:
df["Embarked"] = df["Embarked"].fillna("S")
# тут ваш код c оцифровкой
# df = df[pd.notnull(df['Embarked'])]

df.loc[df['Embarked'] == 'S', 'Embarked'] = 0
df.loc[df['Embarked'] == 'C', 'Embarked'] = 1
df.loc[df['Embarked'] == 'Q', 'Embarked'] = 2
print(df["Embarked"].unique())

[0 1 2]


Разобьем выборку на `X` и `y`, где для`y` будем строить предсказания

In [74]:
print(df.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [75]:
y = pd.Series(df['Survived'])

In [76]:

df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
X = df.copy()
X.drop('Survived', axis = 1, inplace=True)

'''
X.drop('Survived', axis=1, inplace=True)
X.drop('Name', axis=1, inplace=True)
X.drop('Ticket', axis=1, inplace=True)
X.drop('Cabin', axis=1, inplace=True)
'''

"\nX.drop('Survived', axis=1, inplace=True)\nX.drop('Name', axis=1, inplace=True)\nX.drop('Ticket', axis=1, inplace=True)\nX.drop('Cabin', axis=1, inplace=True)\n"

Каким образом можно отобрать фичи, которые будут влиять на наш скор? Тут нам помогут ваш жизненный опыт и ` Recursive Feature Elimination` из библиотеки `Sklearn` 

In [77]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression, LinearRegression
model = LinearRegression()
#тут ваш код

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model.fit(x_train, y_train)

Z = model.predict(x_test)
print(mean_absolute_error(Z, y_test))
print(mean_squared_error(Z, y_test))

0.286438181234
0.1328524492


Проверьте, похожи ли отобранные вами фичи на те, что представленны ниже?

In [78]:
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

In [79]:
kfold = KFold(random_state=1)
X = df[df.columns[2:]].values

Выполним предсказания:

In [80]:
predicts = []
for train, test in kfold.split(X):
    #тут ваш код
    model.fit(X[train], y[train])
    predicts.append(model.predict(X[test]))
predicts = np.concatenate(predicts)

#print (predicts)
print(predicts)

predicts[predicts > 0.5],predicts[predicts < 0.5] = 1,0


accuracy = np.count_nonzero(df["Survived"] == predicts)/df["Survived"].count()
print ('Скор для линейной регрессии:',accuracy)


[ 0.15787598  0.81697641  0.7364616   0.77112227  0.17680262  0.21768048
  0.23203556  0.08618961  0.63185549  0.77801143  0.70274836  0.70922536
  0.20860228 -0.09948925  0.76174698  0.69258599  0.08414442  0.20243361
  0.70308317  0.74326958  0.21593414  0.18971374  0.78515641  0.25148425
  0.61432143  0.44372767  0.20241789  0.5228133   0.7572697   0.1913063
  0.2216596   1.0021961   0.75698804  0.1164244   0.32105957  0.21303272
  0.20242705  0.2064823   0.68590055  0.73696345  0.66541863  0.71810317
  0.20388026  0.71693951  0.7763495   0.19164246  0.18828926  0.75698804
  0.14444381  0.73020687  0.07142318  0.2059373   0.80552888  0.72476336
  0.1886043   0.25148425  0.75267506  0.20136706  0.66997754 -0.02081576
  0.21514691  0.86814744  0.27528919  0.04208819  0.24709933  0.12042098
  0.73571524  0.21095836  0.46709177  0.10773303  0.18850362  0.50943604
  0.34916519  0.17767539  0.28877579  0.19713038  0.1913063   0.19164246
  0.18543353  0.73790082  0.20643335  0.19268353  0.

## Задание 2 (2 балла)
Теперь попробуем проделать то же самое для **логистической регрессии**

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [81]:
model = LogisticRegression()

Вычислим оценку точности для всех кросс-валидационных фолдов: возвращаем массив баллов из 3-х фолдов

In [82]:
#тут ваш код

print("Скор для логистической регрессии: " + str(scores.mean()))

NameError: name 'scores' is not defined

###### Как бы вы объяснили прирост качества в логистической регрессии? 
Сможете побить наш бейзлайн? :)

## Задание 3 (1.5 балла)
Отправьте ваше решение в тестирующую систему и посмотрите, какое место вы займете

рассмотрим `test.csv`

In [ ]:
df_test = pandas.read_csv("test.csv")
print(df_test.describe())

Надо вернуть оцифрованные данные к первоначальному виду:

In [ ]:
#тут ваш код по возвращению оцифрованных параметров в первоначальный вид

df_test["Fare"] = df_test["Fare"].fillna(df_test["Fare"].median())

model.fit(#тут ваш код)
predicts = model.predict(#тут ваш код)

submission = pandas.DataFrame({
    "PassengerId": df_test["PassengerId"],
    "Survived": predicts
})

submission.to_csv("submission.csv", index=False) #вот это отправляется в каггл

Можете попробовать и дальше "поиграться" с параметрами, чтобы подняться в лидерборде. Например, попробовать подобрать параметры в регрессорах. 
Удачи!
    Напоминаем, что справку по любой функции (в том числе и по регрессорам) можно получить, если с наведенным курсором на функцию нажать `Tab`+`Shift`